In [2]:
import pandas as pd
from sklearn.metrics import log_loss

In [75]:
df = pd.read_csv('../data/results/botbrnlys-last-all--2019-02-06.csv')
df.loc[:, 'hp'] = df['hp'].astype(str)
df.loc[:, 'mod_hp'] = df['model'] + df['hp']
df.drop(columns=['hp', 'df_len'], inplace=True)

In [76]:
df.sort_values(by=['mean_log_lss'], ascending=True).iloc[0:10]

,cols,max_accu,max_auc_sc,max_bal_acc,max_brier_lss,max_f_acc,max_log_lss,mdn_accu,mdn_auc_sc,mdn_bal_acc,...,min_log_lss,model,std_accu,std_auc_sc,std_bal_acc,std_brier_lss,std_f_acc,std_log_lss,win_perc,mod_hp
54953,"['diff', 'game_time', 'game_num']",0.7407,0.8098,0.7427,0.2147,0.7538,0.6235,0.6931,0.7593,0.6940,...,0.5394,knn,0.038255,0.028318,0.039217,0.010069,0.041825,0.023553,0.513,knn39.0
54512,"['diff', 'game_time', 'game_num']",0.7460,0.8127,0.7475,0.2167,0.7551,0.6295,0.6984,0.7576,0.6983,...,0.5379,knn,0.038170,0.028722,0.038142,0.010285,0.043146,0.024038,0.484,knn38.0
53630,"['diff', 'game_time', 'game_num']",0.7672,0.8087,0.7673,0.2175,0.7634,0.6317,0.6931,0.7548,0.6934,...,0.5411,knn,0.042427,0.029294,0.042396,0.010355,0.047685,0.024258,0.483,knn36.0
54071,"['diff', 'game_time', 'game_num']",0.7460,0.8100,0.7470,0.2180,0.7562,0.6330,0.6984,0.7584,0.6981,...,0.5400,knn,0.038872,0.029112,0.039806,0.010366,0.043622,0.024412,0.517,knn37.0
54761,"['diff', 'color', 'game_time', 'game_num']",0.7460,0.8052,0.7467,0.2138,0.7500,0.6197,0.6825,0.7515,0.6832,...,0.5432,knn,0.035485,0.025922,0.036308,0.009412,0.040114,0.021554,0.511,knn39.0
54320,"['diff', 'color', 'game_time', 'game_num']",0.7460,0.8076,0.7459,0.2152,0.7461,0.6210,0.6931,0.7557,0.6930,...,0.5413,knn,0.036594,0.028108,0.036549,0.010092,0.040761,0.023137,0.483,knn38.0
53189,"['diff', 'game_time', 'game_num']",0.7513,0.8108,0.7524,0.2171,0.7565,0.6306,0.6878,0.7517,0.6883,...,0.5385,knn,0.040279,0.029117,0.041089,0.010522,0.045295,0.024991,0.521,knn35.0
53879,"['diff', 'color', 'game_time', 'game_num']",0.7566,0.8114,0.7573,0.2164,0.7579,0.6262,0.7037,0.7600,0.7035,...,0.5381,knn,0.040984,0.028775,0.041828,0.010428,0.044145,0.024296,0.518,knn37.0
54983,"['diff', 'game_time', 'weekday']",0.7407,0.7990,0.7418,0.2153,0.7461,0.6248,0.6772,0.7517,0.6774,...,0.5478,knn,0.033032,0.025159,0.033238,0.008897,0.036886,0.020271,0.506,knn39.0
54341,"['diff', 'color', 'game_time', 'weekday', 'gam...",0.7407,0.7989,0.7418,0.2173,0.7461,0.6276,0.6931,0.7553,0.6930,...,0.5476,knn,0.035974,0.026463,0.036211,0.009565,0.044763,0.021447,0.484,knn38.0


In [110]:
gdf = df.groupby(by=['cols', 'mod_hp']).mean().sort_values(by=['mean_accu'], ascending=False).reset_index()

In [111]:
gdf.loc[:, 'mean_rank'] = (gdf.mean_accu.rank(ascending=False, numeric_only=True) +
                           gdf.mean_auc_sc.rank(ascending=False, numeric_only=True) +
                          gdf.mean_bal_acc.rank(ascending=False, numeric_only=True) +
                           gdf.mean_f_acc.rank(ascending=False, numeric_only=True) +
                          gdf.mean_brier_lss.rank(ascending=True, numeric_only=True) +
                           gdf.mean_log_lss.rank(ascending=True, numeric_only=True))/6

In [112]:
gdf.loc[:, 'min_rank'] = (gdf.min_accu.rank(ascending=False, numeric_only=True) +
                          gdf.min_auc_sc.rank(ascending=False, numeric_only=True) +
                          gdf.min_bal_acc.rank(ascending=False, numeric_only=True) +
                          gdf.min_f_acc.rank(ascending=False, numeric_only=True) +
                          gdf.max_brier_lss.rank(ascending=True, numeric_only=True) +
                          gdf.max_log_lss.rank(ascending=True, numeric_only=True))/6

In [113]:
gdf.loc[:, 'max_rank'] = (gdf.max_accu.rank(ascending=False, numeric_only=True) +
                          gdf.max_auc_sc.rank(ascending=False, numeric_only=True) +
                          gdf.max_bal_acc.rank(ascending=False, numeric_only=True) +
                          gdf.max_f_acc.rank(ascending=False, numeric_only=True) +
                          gdf.min_brier_lss.rank(ascending=True, numeric_only=True) +
                          gdf.min_log_lss.rank(ascending=True, numeric_only=True))/6

In [115]:
mx_min = gdf.max_rank.min()
mx_max = gdf.max_rank.max()
av_min = gdf.mean_rank.min()
av_max = gdf.mean_rank.max()
mn_min = gdf.min_rank.min()
mn_max = gdf.min_rank.max()
mx_xn = mx_max - mx_min
av_xn = av_max - av_min
mn_xn = mn_max - mn_min

In [116]:
gdf.loc[:, 'mean_rank'] = (((gdf.mean_rank - av_min)*len(gdf)) / av_xn)
gdf.loc[:, 'min_rank'] = (((gdf.min_rank - mn_min)*len(gdf)) / mn_xn)
gdf.loc[:, 'max_rank'] = (((gdf.max_rank - mx_min)*len(gdf)) / mx_xn)

In [118]:
gdf.sort_values(by=['max_log_lss'], ascending=True).iloc[0:200].to_csv('./mxll.csv')

In [117]:
gdf.sort_values(by='mean_rank')

,cols,mod_hp,max_accu,max_auc_sc,max_bal_acc,max_brier_lss,max_f_acc,max_log_lss,mdn_accu,mdn_auc_sc,...,std_accu,std_auc_sc,std_bal_acc,std_brier_lss,std_f_acc,std_log_lss,win_perc,mean_rank,min_rank,max_rank
0,"['color', 'time', 'elo', 'opp_elo']",logistic_bal2.25,0.774967,0.848333,0.774600,0.239133,0.771867,0.860333,0.688933,0.751767,...,0.050114,0.052004,0.049205,0.017528,0.054903,0.072205,0.481333,0.000000,264.259391,6474.636402
15,"['diff', 'time', 'weekday', 'elo', 'opp_elo']",logistic_bal3.0,0.778533,0.847800,0.777533,0.238667,0.777933,0.859233,0.678400,0.754800,...,0.050478,0.052348,0.049007,0.018179,0.059552,0.072721,0.478000,18.936945,1122.016283,4831.124398
31,"['diff', 'time', 'weekday', 'elo']",logistic_bal3.0,0.771500,0.849367,0.770867,0.241367,0.772033,0.857433,0.674867,0.747700,...,0.052208,0.052814,0.050141,0.018559,0.063710,0.072519,0.480667,63.152925,1597.456516,5983.739053
23,"['diff', 'time', 'elo']",logistic_bal1.0,0.778500,0.846967,0.773700,0.240300,0.784167,0.861600,0.676633,0.751400,...,0.052806,0.052574,0.051283,0.018116,0.064047,0.072978,0.477667,66.100657,1645.246099,4564.906630
9,"['diff', 'color', 'game_time', 'elo']",logistic_bal1.25,0.787267,0.850267,0.779933,0.235833,0.783500,0.896233,0.687167,0.756867,...,0.048251,0.048212,0.047744,0.016822,0.059106,0.077991,0.490667,113.800320,99.168106,2908.886498
45,"['diff', 'time', 'elo']",logistic_bal0.5,0.778500,0.846533,0.773067,0.242733,0.781033,0.861633,0.676633,0.751400,...,0.053594,0.052081,0.051218,0.018535,0.067839,0.073336,0.483667,118.177255,2366.623234,5131.599501
62,"['diff', 'time', 'elo']",logistic_bal0.25,0.778500,0.846533,0.773067,0.242733,0.771233,0.861633,0.676633,0.751400,...,0.052910,0.052122,0.050603,0.018552,0.066825,0.073397,0.482000,133.541192,2366.623234,5986.539380
71,"['diff', 'time', 'elo']",logistic_bal-0.25,0.778500,0.846533,0.773067,0.242733,0.771233,0.861633,0.676633,0.751400,...,0.052910,0.052155,0.050584,0.018554,0.066702,0.073385,0.482333,134.077143,2366.623234,5986.539380
66,"['diff', 'time', 'elo']",logistic_bal-0.5,0.778500,0.846533,0.773067,0.242733,0.771233,0.861633,0.676633,0.751400,...,0.052910,0.052155,0.050584,0.018554,0.066702,0.073385,0.482333,134.077143,2366.623234,5986.539380
63,"['diff', 'time', 'elo']",logistic_bal0.0,0.778500,0.846533,0.773067,0.242733,0.771233,0.861633,0.676633,0.751400,...,0.052910,0.052155,0.050584,0.018554,0.066702,0.073385,0.482333,134.077143,2366.623234,5986.539380


In [39]:
df = initial_df('../data/use_for_predictions.csv')
y = df.result.values
X = df[['diff', 'color', 'start_time', 'game_time', 'weekday', 'day_game_num']].values

model = XGBClassifier()
for i in range(0, 2500, 50):
    j = i + 55
    X_ = X[i: j]
    y_ = y[i: j]
    model.fit(X_, y_)

[0.4925816  0.08902077 0.14243324 0.12462908 0.08902077 0.06231454]
[0.48356807 0.07981221 0.16666667 0.         0.13380282 0.13615024]
[0.42154565 0.06791569 0.17564403 0.         0.10070258 0.23419204]
[0.3604651  0.13488372 0.18604651 0.         0.14651163 0.17209302]
[0.38117647 0.05411765 0.22117648 0.         0.12941177 0.21411765]
[0.23940149 0.08478803 0.22194514 0.         0.159601   0.29426435]
[0.4811321  0.10613208 0.13443395 0.         0.07783019 0.2004717 ]
[0.4888393  0.06696428 0.16741072 0.03125    0.12276786 0.12276786]
[0.3847981  0.07125891 0.09976247 0.11163896 0.14726841 0.18527316]
[0.3125     0.0327381  0.1845238  0.05357143 0.27083334 0.14583333]
[0.49753696 0.05172414 0.12807882 0.01970443 0.14039409 0.16256158]
[0.4362745  0.03921569 0.2867647  0.         0.16176471 0.0759804 ]
[0.42004773 0.07875895 0.15990454 0.         0.14797136 0.19331743]
[0.36682242 0.0046729  0.2406542  0.02570093 0.21028037 0.15186916]
[0.45961002 0.06406685 0.13091922 0.01392758 0.1

In [23]:
from pandas import read_csv
import pandas as pd
from numpy import sort
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from dummies_bins_test_train_cv import initial_df
from sklearn.ensemble import AdaBoostClassifier as ABC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import BaggingClassifier as BGC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.gaussian_process import GaussianProcessClassifier as GPC
from sklearn.linear_model import LogisticRegression as LGR
from xgboost import XGBClassifier as XGB
from sklearn.linear_model import RidgeClassifier as RDC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.svm import NuSVC as NVC
from sklearn.svm import SVC
import datetime

In [3]:
all_models = ['abc', 'rfc', 'bgc', 'gbc', 'gpc', 'lgr',
              'xgb', 'rdc', 'knc', 'nvc', 'svc']

In [20]:
def get_model(clf):
    if clf == 'abc':
        model = ABC(base_estimator=RFC(n_estimators=300), random_state=5)
    elif clf == 'rfc':
        model = RFC(n_estimators=300, random_state=5)
    elif clf == 'bgc':
        model = BGC(n_estimators=300, random_state=5)
    elif clf == 'gbc':
        model = GBC(max_depth=200, random_state=5)
    elif clf == 'gpc':
        model = GPC(n_restarts_optimizer=5, random_state=5)
    elif clf == 'lgr':
        model = LGR(solver='newton-cg', random_state=5)
    elif clf == 'xgb':
        model = XGB(max_depth=200, random_state=5)
    elif clf == 'rdc':
        model = RDC(random_state=5)
    elif clf == 'knc':
        model = KNC(n_neighbors=33)
    elif clf == 'nvc':
        model = NVC(gamma='scale', random_state=5, probability=True)
    elif clf == 'svc':
        model = SVC(gamma='scale', random_state=5, probability=True)
    return model

In [21]:
def run_model(X, y, all_models, result_df):
    ind_num = 0
    d_t = datetime.datetime.now().time()
    fn = f'../data/mui_results-{d_t}.csv'
    for clf in all_models:
        print(clf, ind_num)
        model = get_model(clf)
        for i in range(0, len(X), 500):
            ind_num += 1
            i = (len(X) - 601) if (i + 601 >= len(X)) else i
            j = i + 501
            k = j + 100
            result_df.loc[ind_num, 'classifier'] = clf
            result_df.loc[ind_num, 'train_start_row'] = i
            result_df.loc[ind_num, 'train_end_row'] = j-1
            result_df.loc[ind_num, 'test_start_row'] = j
            result_df.loc[ind_num, 'test_end_row'] = k-1

            X_train = X[i: j]
            y_train = y[i: j]
            X_test = X[j: k]
            y_test = y[j: k]
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_pred = [1 if val > 0.5 else 0 for val in y_pred]
            acc = accuracy_score(y_test, y_pred)
            result_df.loc[ind_num, 'accuracy'] = round(acc*100, 4)
            print(f'Train: {i}-{j-1} -|- Test: {j}-{k-1} -|- {round(acc*100, 2)}')
            result_df.to_csv(fn)
            try:
                mfi = model.feature_importances_.round(4)
                result_df.loc[ind_num, 'mfi'] = str(mfi)
                print(mfi)
                thresholds = sort(model.feature_importances_)
                for thresh in thresholds:
                    selection = SelectFromModel(model, threshold=thresh-1e-10, prefit=True)
                    try:
                        select_X_train = selection.transform(X_train)
                        numba = select_X_train.shape[1]
                        if numba >= 1 and len(select_X_train) > 0:
                            selection_model = get_model(clf)
                            selection_model.fit(select_X_train, y_train)
                            select_X_test = selection.transform(X_test)
                            y_pred = selection_model.predict(select_X_test)
                            y_pred = [1 if val > 0.5 else 0 for val in y_pred]
                            acc = accuracy_score(y_test, y_pred)
                            result_df.loc[ind_num, f'n_{numba}'] = round(acc*100, 4)
                            print(f'Thresh={thresh}, n={numba}, Accuracy: {acc*100}')
                            result_df.to_csv(fn)
                        else:
                            pass
                    except TypeError:
                        pass
            except AttributeError:
                pass

In [22]:
df = initial_df('../data/use_for_predictions.csv')
df.loc[:, 'color'] = df.color * 0.083617571059432
df.loc[:, 'game_time'] = df.game_time * 0.082721791559001
df.loc[:, 'day_game_num'] = df.day_game_num * 0.379810508182601
df.loc[:, 'weekday'] = df.weekday * 0.213264427217916
df.loc[:, 'start_time'] = df.start_time * 0.386701119724375
df.loc[:, 'elo'] = df.elo * 0.740361757105943
df.loc[:, 'opp_elo'] = df.opp_elo * 0.903359173126615
y = df.result.values
X = df[['diff', 'start_time', 'weekday', 'day_game_num', 'game_time', 'color', 'elo', 'opp_elo']].values
result_df = pd.DataFrame()
run_model(X, y, all_models, result_df)

abc 0
Train: 0-500 -|- Test: 501-600 -|- 66.0
[0.2726 0.1164 0.0732 0.1047 0.0192 0.0312 0.1627 0.2199]
Thresh=0.019206849535019995, n=8, Accuracy: 66.0
Thresh=0.0311970332044519, n=7, Accuracy: 66.0
Thresh=0.0732212605178343, n=6, Accuracy: 63.0
Thresh=0.1046831056609826, n=5, Accuracy: 63.0
Thresh=0.11644487060760561, n=4, Accuracy: 64.0
Thresh=0.1627387813692489, n=3, Accuracy: 65.0
Thresh=0.21990060339413195, n=2, Accuracy: 68.0
Thresh=0.2726074957107248, n=1, Accuracy: 56.00000000000001
Train: 500-1000 -|- Test: 1001-1100 -|- 70.0
[0.2555 0.1301 0.0839 0.0912 0.0343 0.0281 0.1723 0.2046]
Thresh=0.02809455598576548, n=8, Accuracy: 70.0
Thresh=0.03427777392296734, n=7, Accuracy: 65.0
Thresh=0.08390351022707489, n=6, Accuracy: 62.0
Thresh=0.0912203718706342, n=5, Accuracy: 63.0
Thresh=0.13007254251195277, n=4, Accuracy: 63.0
Thresh=0.17233143089909755, n=3, Accuracy: 63.0
Thresh=0.20458841595912772, n=2, Accuracy: 56.99999999999999
Thresh=0.2555113986233803, n=1, Accuracy: 64.0
Train

Train: 1908-2408 -|- Test: 2409-2508 -|- 57.0
[0.3544 0.1255 0.1104 0.1111 0.0085 0.0336 0.1289 0.1277]
Thresh=0.008489766676058726, n=8, Accuracy: 56.99999999999999
Thresh=0.03360612198254457, n=7, Accuracy: 56.00000000000001
Thresh=0.11036868755934189, n=6, Accuracy: 48.0
Thresh=0.11107044671037428, n=5, Accuracy: 51.0
Thresh=0.12547060374268176, n=4, Accuracy: 55.00000000000001
Thresh=0.1277034104534479, n=3, Accuracy: 61.0
Thresh=0.12889550211774145, n=2, Accuracy: 63.0
Thresh=0.3543954607578094, n=1, Accuracy: 61.0
Train: 1908-2408 -|- Test: 2409-2508 -|- 57.0
[0.3544 0.1255 0.1104 0.1111 0.0085 0.0336 0.1289 0.1277]
Thresh=0.008489766676058726, n=8, Accuracy: 56.99999999999999
Thresh=0.03360612198254457, n=7, Accuracy: 56.00000000000001
Thresh=0.11036868755934189, n=6, Accuracy: 48.0
Thresh=0.11107044671037428, n=5, Accuracy: 51.0
Thresh=0.12547060374268176, n=4, Accuracy: 55.00000000000001
Thresh=0.1277034104534479, n=3, Accuracy: 61.0
Thresh=0.12889550211774145, n=2, Accuracy: 

In [ ]:
col_list = ['diff', 'color', 'time', 'game_time', 'weekday', 'elo', 'opp_elo', 'game_num', 'gpiltd', 'gps']

In [ ]:
combinations(iterable, r)

196.5